In [6]:

import os, shutil
from glob import glob
import numpy as np
import pandas as pd
from natsort import natsorted
import seaborn as sns
from tqdm import tqdm, trange
import matplotlib.pyplot as plt


In [7]:

in_dir = "out01_tms_induced_respose/"
in_dir2 = "data_subject_info/"

out_dir = "out02_multiple_regression/"
in_file_name = "HRF_tms_induced_response.csv"

if not os.path.exists(out_dir):
    os.mkdir(out_dir)
    
complete_df = pd.read_csv(in_dir + in_file_name)

subject_info = pd.read_csv(in_dir2 + "age_gender_edu.txt", sep = ',')
intensity_suds = pd.read_csv(in_dir2 + "intensity_suds.csv", sep = ',')
scalp_dist = pd.read_csv(in_dir2 + "dist_to_scalp.csv", sep = ',')
print(list(scalp_dist['site'].drop_duplicates()))


['L-FP', 'R-FP', 'L-aMFG', 'R-aMFG', 'L-pMFG', 'R-pMFG', 'R-IFJ', 'R-FEF', 'R-M1', 'R-preSMA', 'R-IPL']


In [8]:
# scalp_dist['site'] = scalp_dist['site'].str.replace('-', '_')
# df['range'].str.replace(',','-')

tms_sites = ["L_Fp","R_Fp","L_aMFG","R_aMFG","L_pMFG","R_pMFG","R_IFJ","R_FEF","R_M1","R_preSMA","R_IPL"]
scalp_dist['site'].replace(list(scalp_dist['site'].drop_duplicates()), tms_sites, inplace = True)

# reformat columns in intensity_suds:
intensity_suds = pd.melt(intensity_suds, id_vars=['idall', 'MT', 'intensity'], value_vars = tms_sites,
                         var_name = 'suds_site', value_name = 'suds')

data = subject_info.merge(complete_df, left_on = 'cc_post_intake_id', right_on = 'subject')
data = data.merge(intensity_suds, how = 'left', 
                  left_on = ['cc_post_intake_id', 'site'], right_on = ['idall', 'suds_site'])

data = data.merge(scalp_dist, how = 'left', 
                  left_on = ['subject', 'site'], right_on = ['subject', 'site'])

data.drop(labels = ['inputfile', 'idall', 'cc_post_intake_id', 'suds_site'], axis = 1, inplace = True)
# data.replace({'gender': {1: "male", 2: "female"}}, inplace = True)
# data.replace({'gender': {'male': 1, 'female': 2}}, inplace = True)

data


,gender,age,yrs_of_edu,subject,site,group,tms_site_response_standard,tms_site_response_individual_6mm,tms_site_response_individual_10mm,tms_site_response_individual_14mm,tms_site_response_individual_14-10mm,tms_site_response_individual_10-6mm,MT,intensity,suds,scalp_dist
0,2,45,18,1001,L_Fp,NTHC,0.287125,0.439684,0.456288,0.395807,0.354767,0.462649,62.0,74.0,10.0,14.749354
1,2,45,18,1001,L_pMFG,NTHC,-0.358279,-0.463813,-0.439631,-0.329946,-0.264645,-0.431570,62.0,74.0,5.0,13.038824
2,2,45,18,1001,R_FEF,NTHC,-0.252532,-0.423779,-0.511036,-0.496599,-0.487678,-0.540984,62.0,74.0,2.0,17.079627
3,2,45,18,1001,R_Fp,NTHC,0.245202,0.278052,0.265166,0.243850,0.230372,0.260594,62.0,74.0,7.0,16.614293
4,2,45,18,1001,R_M1,NTHC,-1.002989,-1.403756,-1.606550,-1.550770,-1.520768,-1.663231,62.0,74.0,1.0,15.276347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,1,22,14,2108,R_IPL,TEHC,1.236482,0.391417,1.001566,1.383794,1.611196,1.196951,NaN,100.0,5.0,14.361543
788,1,22,14,2108,R_M1,TEHC,0.124382,0.233850,0.166974,0.150504,0.140419,0.144036,NaN,100.0,15.0,18.975313
789,1,22,14,2108,R_aMFG,TEHC,-0.279814,-0.045608,-0.032161,-0.082834,-0.115168,-0.027050,NaN,100.0,20.0,13.567683
790,1,22,14,2108,R_pMFG,TEHC,0.363362,-0.020677,0.119496,0.184963,0.224639,0.165267,NaN,100.0,20.0,14.366030


In [4]:

## correlation matrix:
d = data.loc[:, ['gender', 'age', 'yrs_of_edu', 'MT', 'intensity', 'suds', 'scalp_dist']]
print(d.count())
d.corr()


gender        792
age           792
yrs_of_edu    792
MT            581
intensity     792
suds          771
scalp_dist    689
dtype: int64


,gender,age,yrs_of_edu,MT,intensity,suds,scalp_dist
gender,1.000000,0.100431,0.030555,0.026028,-0.005011,-0.027004,0.086767
age,0.100431,1.000000,0.430420,-0.219282,-0.164505,0.079981,0.078526
yrs_of_edu,0.030555,0.430420,1.000000,-0.085059,-0.087906,-0.001968,0.044035
MT,0.026028,-0.219282,-0.085059,1.000000,0.999374,0.011761,0.134476
intensity,-0.005011,-0.164505,-0.087906,0.999374,1.000000,0.114597,0.199412
suds,-0.027004,0.079981,-0.001968,0.011761,0.114597,1.000000,-0.125881
scalp_dist,0.086767,0.078526,0.044035,0.134476,0.199412,-0.125881,1.000000


In [5]:
import statsmodels
statsmodels.__version__

'0.13.5'

In [6]:
import statsmodels.api as sm

# backward selection

test_variables = ['tms_site_response_standard', 'tms_site_response_individual_6mm',
                  'tms_site_response_individual_10mm', 'tms_site_response_individual_14mm', 
                  'tms_site_response_individual_14-10mm', 'tms_site_response_individual_10-6mm']

roi_list = data.site.unique()
feature = ['gender', 'age', 'yrs_of_edu', 'intensity', 'suds', 'scalp_dist']

row_index = pd.MultiIndex.from_tuples([(i , j) for i in test_variables for j in feature])
result = pd.DataFrame(index = row_index, columns = tms_sites)

for var in test_variables:    
    for roi in roi_list:
        print(var)
        print(roi)
        
        X = data.loc[(data['site'] == roi) & (~data[var].isna()), feature]
        y = data.loc[(data['site'] == roi) & (~data[var].isna()), [var]]

        if len(y) < 20: continue
            
        # remove nan in X:
        row_nan = X.isna().any(axis=1)
        X = sm.add_constant(X.loc[~row_nan,:]) # adding a constant

        model = sm.OLS(y.loc[~row_nan], X).fit()
        predictions = model.predict(X) 

#         print(model.summary())
        for i, f in enumerate(feature):
            result.loc[(var, f), roi] = model.pvalues[i]        


tms_site_response_standard
L_Fp
tms_site_response_standard
L_pMFG
tms_site_response_standard
R_FEF
tms_site_response_standard
R_Fp
tms_site_response_standard
R_M1
tms_site_response_standard
R_aMFG
tms_site_response_standard
R_pMFG
tms_site_response_standard
L_aMFG
tms_site_response_standard
R_IFJ
tms_site_response_standard
R_preSMA
tms_site_response_standard
R_IPL
tms_site_response_individual_6mm
L_Fp
tms_site_response_individual_6mm
L_pMFG
tms_site_response_individual_6mm
R_FEF
tms_site_response_individual_6mm
R_Fp
tms_site_response_individual_6mm
R_M1
tms_site_response_individual_6mm
R_aMFG
tms_site_response_individual_6mm
R_pMFG
tms_site_response_individual_6mm
L_aMFG
tms_site_response_individual_6mm
R_IFJ
tms_site_response_individual_6mm
R_preSMA
tms_site_response_individual_6mm
R_IPL
tms_site_response_individual_10mm
L_Fp
tms_site_response_individual_10mm
L_pMFG
tms_site_response_individual_10mm
R_FEF
tms_site_response_individual_10mm
R_Fp
tms_site_response_individual_10mm
R_M1
tm

In [7]:

def format_table(report):
    report2 = report.astype(str).apply(lambda x : x.str[:5])
    
    report2[report.le(0.05)] = \
    report2[report.le(0.05)].apply(lambda x : x.str[:5]).add('*')

    report2[report.le(0.01)] = \
    report2[report.le(0.01)].apply(lambda x : x.str[:5]).add('**')
        
    return report2

result2 = format_table(result)
result2.to_csv(out_dir + "multiple_regression_pvalues_all.csv")

for var in test_variables:
    result2.loc[(var, feature),:].to_csv(out_dir + "multiple_regression_pvalues_" + var + ".csv")

result2


L_Fp   R_Fp   L_aMFG  \
tms_site_response_standard           gender        0.076  0.873    0.498   
                                     age           0.404  0.533  0.009**   
                                     yrs_of_edu    0.696  0.737    0.088   
                                     intensity     0.290  0.708    0.249   
                                     suds          0.302  0.916    0.155   
                                     scalp_dist    0.314  0.799    0.775   
tms_site_response_individual_6mm     gender       0.025*  0.970    0.269   
                                     age           0.546  0.930    0.145   
                                     yrs_of_edu    0.155  0.347   0.028*   
                                     intensity     0.258  0.993    0.261   
                                     suds          0.052  0.902   0.047*   
                                     scalp_dist  0.003**  0.788    0.714   
tms_site_response_individual_10mm    gender        0.056  0.745    0.256   
                                     age           0.262  0.681    0.053   
                                     yrs_of_edu    0.389  0.699   0.041*   
                                     intensity     0.133  0.731    0.198   
                                     suds          0.175  0.924   0.033*   
                                     scalp_dist  0.005**  0.969    0.828   
tms_site_response_individual_14mm    gender        0.078  0.826    0.337   
                                     age           0.315  0.660   0.017*   
                                     yrs_of_edu    0.606  0.929    0.068   
                                     intensity     0.095  0.690    0.189   
                                     suds          0.276  0.976   0.043*   
                                     scalp_dist  0.008**  0.934    0.954   
tms_site_response_individual_14-10mm gender        0.108  0.855    0.411   
                                     age           0.333  0.676   0.011*   
                                     yrs_of_edu    0.722  0.962    0.107   
                                     intensity     0.094  0.691    0.207   
                                     suds          0.362  0.959    0.058   
                                     scalp_dist   0.012*  0.924    0.803   
tms_site_response_individual_10-6mm  gender        0.074  0.734    0.253   
                                     age           0.217  0.668   0.041*   
                                     yrs_of_edu    0.450  0.696    0.054   
                                     intensity     0.124  0.730    0.193   
                                     suds          0.229  0.916   0.031*   
                                     scalp_dist  0.006**  0.981    0.883   

                                                R_aMFG L_pMFG R_pMFG   R_IFJ  \
tms_site_response_standard           gender      0.051  0.590  0.761   0.107   
                                     age         0.273  0.721  0.118   0.740   
                                     yrs_of_edu  0.337  0.716  0.307   0.901   
                                     intensity   0.247  0.535  0.454   0.696   
                                     suds        0.773  0.551  0.994   0.079   
                                     scalp_dist  0.714  0.866  0.739   0.225   
tms_site_response_individual_6mm     gender      0.904  0.551  0.938   0.397   
                                     age         0.889  0.767  0.577   0.468   
                                     yrs_of_edu  0.512  0.833  0.714   0.359   
                                     intensity   0.903  0.269  0.749   0.425   
                                     suds        0.554  0.571  0.824   0.051   
                                     scalp_dist  0.315  0.265  0.464   0.305   
tms_site_response_individual_10mm    gender      0.488  0.476  0.987   0.114   
                                     age         0.681  0.929  0.419   0.539   
                                     yr